In [1]:
import RAPIDpy

In [2]:
from imports import *
import time

from torch_function import block_sparse_conv_1d_autograd
from gen_data import generate_inputs

sys.path.append("..")
from jphydro.routing.block_sparse_conv import BlockSparseCausalConv
from jphydro.routing.kernel_aggregator_irf import RoutingIRFAggregator

In [3]:
n_trees = 1
max_heights = 3
n_time_steps = 365
max_delay = 10
block_size = 32
batch_size = 1
BLOCK_SIZE_N = 64
device = "cuda:0"
BLOCK_SIZE_M = block_size

In [4]:
g, x, k = generate_inputs(n_trees, max_heights, max_delay, block_size, n_time_steps, batch_size)

In [18]:
df_runoff = pd.DataFrame(x.cpu().detach().numpy()[0]).T.set_index(pd.date_range("2008-1-1","2008-12-30"))

In [ ]:
import networkx as nx
import pandas as pd

def save_graph_to_csv(g, network_file='rapid_river_network.csv', conn_file='rapid_connectivity.csv'):
    """
    Saves a directed acyclic graph (DAG) in a format that can be used
    later for RAPID param files (this example is simplistic).
    
    - network_file: CSV with columns [reach_id].
    - conn_file: CSV with columns [reach_id, downstream_id].
    """
    # 1. Get a topological ordering of the graph
    topo_order = list(nx.topological_sort(g))
    
    # 2. Assign each reach a 1-based index for RAPID
    #    (common convention in the RAPID code)
    reach_idx_map = {rid: i+1 for i, rid in enumerate(topo_order)}
    
    # 3. Write a CSV with just the reach list in topological order
    df_network = pd.DataFrame({"reach_id": [reach_idx_map[r] for r in topo_order]})
    df_network.to_csv(network_file, index=False)
    
    # 4. Create a CSV for connectivity (who flows to whom)
    #    For nodes with no downstream, use 0 or -9999 based on your RAPID config
    conn_data = []
    for r in topo_order:
        successors = list(g.successors(r))
        if len(successors) == 0:
            ds_id = 0  # or -9999
        else:
            # if there's exactly one downstream
            ds_id = reach_idx_map[successors[0]]
        conn_data.append({"reach_id": reach_idx_map[r], "downstream_id": ds_id})
    
    df_conn = pd.DataFrame(conn_data)
    df_conn.to_csv(conn_file, index=False)

    # Return the topo_order and mapping so we can use it in other steps
    return topo_order, reach_idx_map


import netCDF4 as nc
import numpy as np

def save_runoff_to_netcdf(df_runoff, reach_idx_map, netcdf_file='rapid_runoff_in.nc'):
    """
    Saves runoff data to a disk NetCDF file in a shape that RAPID can understand:
    Dimensions: time, rivid
    Variable: m3_riv (units are typically m^3/s or something similar)
    
    - df_runoff: DataFrame with DatetimeIndex and columns for each node in the graph
    - reach_idx_map: dict mapping from node_label -> 1-based rivid index
    """
    # 1. Convert times
    times = df_runoff.index.to_pydatetime()
    
    # 2. Sort columns by the topological order or by reach_idx
    #    So that we write them in ascending rivid order
    #    In practice, you must ensure the same ordering that RAPID expects.
    sorted_cols = sorted(df_runoff.columns, key=lambda c: reach_idx_map[c])
    
    # 3. Create the netCDF file
    ds = nc.Dataset(netcdf_file, mode='w', format='NETCDF4')
    
    # Dimensions
    ds.createDimension('time', len(times))
    ds.createDimension('rivid', len(sorted_cols))
    
    # Create variables
    time_var = ds.createVariable('time', 'f8', ('time',))
    rivid_var = ds.createVariable('rivid', 'i4', ('rivid',))
    m3_var = ds.createVariable('m3_riv', 'f4', ('time','rivid',), zlib=True)
    
    # Write time data (simple numeric - seconds from a reference epoch)
    # Alternatively, you can store them in a CF-compliant manner
    import datetime as dt
    ref_time = dt.datetime(1970,1,1)
    time_var[:] = [(t - ref_time).total_seconds() for t in times]
    time_var.units = f'seconds since {ref_time.strftime("%Y-%m-%d %H:%M:%S")}'
    
    # Write rivid
    rivid_var.long_name = "unique reach identifier"
    # We place them in ascending order
    rivid_sorted = [reach_idx_map[c] for c in sorted_cols]
    rivid_var[:] = rivid_sorted
    
    # Write runoff data
    # shape => [time, rivid]
    # so we take the columns in sorted_cols
    arr_runoff = df_runoff[sorted_cols].values
    m3_var[:, :] = arr_runoff
    
    ds.close()

# Usage
# df_runoff = ...


In [24]:
topo_order, reach_idx_map = save_graph_to_csv(g)
save_runoff_to_netcdf(df_runoff, reach_idx_map, netcdf_file='rapid_runoff_in.nc')

In [26]:
import time
from RAPIDpy import RAPID

def run_rapid_simulation(
    rapid_executable,
    river_network_csv='rapid_river_network.csv',
    connectivity_csv='rapid_connectivity.csv',
    runoff_nc='rapid_runoff_in.nc',
    qout_nc='rapid_qout.nc'
):
    """
    Example function to configure and run RAPID with 
    file-based input for the connectivity and runoff data.
    """
    # Initialize the RAPID manager
    rapid_manager = RAPID(
        rapid_executable=rapid_executable,
        # Typically you'd specify parameter files here.
        # For example:
        qout_file=qout_nc,
    )
    
    # Update the manager's parameters for input data
    # This can vary depending on how your `rapidpy` is set up.
    # Some possible calls:
    rapid_manager.update_parameters(
        inflow_file=runoff_nc,  # NetCDF with inflow
        # Example: pass CSVs or create a RAPID input parameter file from them
        # This might require a separate step. For example, some versions
        # of rapipy have a method to read CSV connectivity or to build
        # a "rapid_connectivity" file automatically.
        
        # If your version of RAPID requires a "rapid_connect_file" in netcdf:
        # you might need to build that from the CSV. 
        # We'll assume you have a method:
        # 'connectivity_csv_to_rapid_params(connectivity_csv)' etc.

        # placeholders:
        rapid_connect_file='rapid_connectivity.nc',  # if needed
        # or some other approach
    )
    
    # Actually run the model (calls the Fortran code)
    t0 = time.perf_counter()
    rapid_manager.run()
    t1 = time.perf_counter()
    
    print(f"RAPID simulation finished in {t1 - t0:.3f} seconds.")
    

In [27]:
run_rapid_simulation("/data_prediction005/SYSTEM/prediction002/home/tristan/repo/rapid/src/rapid")

Exception: ERROR: Invalid RAPID parameter rapid_executable.